In [ ]:
#install.packages("R.matlab")
#install.packages("ggplot2")
#install.packages("gridextra")
#install.packages("plotly")
#devtools::install_github('GRousselet/rogme')

In [ ]:
library(rogme)
library(tibble)
library(ggplot2)
library(R.matlab)
library(plotly)
library(htmlwidgets)
#library(cowplot)

In [ ]:
install.packages(feather)

In [ ]:
percentage_difference <- function(value, value_two) {
  pctd = abs(value - value_two) / ((value + value_two) / 2) * 100
  return(pctd)
} 

In [ ]:
set.seed(123)

derivativesDir <- "/Users/agah/Desktop/KuzuData/PHD_DATA/ds-venus/derivatives/qMRLab/sub-invivo/"
sessions <- list("ses-rth750retest", "ses-rth750test","ses-rthPRIretest","ses-rthPRItest","ses-rthSKYretest","ses-rthSKYtest","ses-vendor750retest", "ses-vendor750test","ses-vendorPRIretest","ses-vendorPRItest","ses-vendorSKYretest","ses-vendorSKYtest")
sessionsB1 <- list("ses-rth750retest", "ses-rth750test","ses-rthPRIretest","ses-rthPRItest","ses-rthSKYretest","ses-rthSKYtest","ses-vendorPRIretest","ses-vendorPRItest","ses-vendorSKYretest","ses-vendorSKYtest")


## T1 without correction

In [ ]:
df <- tibble(N = 1:50000)
for (i in seq(from=1, to=length(sessions), by=2)) {
  curRetest <- readMat(paste(derivativesDir,sessions[i],"/stat/sub-invivo_",sessions[i],"_desc-WM_metrics.mat",sep = ""))
  curTest <- readMat(paste(derivativesDir,sessions[i+1],"/stat/sub-invivo_",sessions[i+1],"_desc-WM_metrics.mat",sep = ""))
  tmp <- Map(function(x,y) mean(c(x,y)), curRetest$"T1", curTest$"T1")
  curAvg <- as.numeric(unlist(tmp))
  curAvg <- curAvg[curAvg > 0]
  curAvg <- curAvg[curAvg < 3]
  # Do not choose the same element more than once            
  curAvg <- sample (curAvg, size=50000, replace =F)          
  varName <- toString(sessions[i])
  varName <- substr(varName,5,nchar(varName)-6)
  df <- mutate(df, !!varName := curAvg)       
}
             
neutral = list("rth750","rthPRI","rthSKY")
native  = list("vendor750","vendorPRI","vendorSKY")
combolar <- combn(c(1,2,3),2)

for (j in seq(from=1,to=2, by=1)){
for (i in seq(from=1,to=3, by=1)){

if (j==1){
    sel1 = toString(neutral[combolar[,i][1]])
    sel2 = toString(neutral[combolar[,i][2]])
    }
else
{
    sel1 = toString(native[combolar[,i][1]])
    sel2 = toString(native[combolar[,i][2]])
}

paste(sel1,"vs",sel2," T1",' in progress',sep="")
    
xx <- df[[sel1]]
yy <- df[[sel2]]

aa <- mkt2(xx,yy,group_labels=c(sel1,sel2))
sf <- shiftdhd_pbci(
  data = aa,
  formula = obs ~ gr,
  nboot = 250,
  alpha = 0.05/30,
)

sf[[1]]['pctdif'] = percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])
write.csv(sf,paste(sel1,"vs",sel2,"_T1_22",'.csv',sep=""))
print(min(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))
print(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])[5])
print(max(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))

    
th <- ggplot2::theme_dark() + ggplot2::theme(
  # get rid of panel grids
  #panel.grid.major = element_blank(),
  #panel.grid.minor = element_blank(),
 # legend.title = element_blank(),  
  # Change plot and panel background
  panel.border = element_blank(),
  panel.background = element_rect(fill = "black"),
  plot.background = element_rect(fill = "black"),
  legend.background = element_rect(fill="white",size=0.5),  
  legend.text=element_text(color="white",size=12),  
  #panel.border=element_blank(),   
  axis.line=element_blank(),
  #axis.ticks = element_blank(),
  axis.text = element_blank(),
    axis.title = element_blank(),
  #axis.title = element_text(colour="white",size=8),
  axis.text.x = element_text(colour="white",size=18,face='bold'),
  axis.text.y = element_text(colour="white",size=18,face='bold')
  ) 


if (j==1){
    lnclr = "darkgray"
    flclr = "red"
} else
{
    lnclr = "darkgray"
    flclr = "blue"
}
    
psf <- plot_sf(
  data = sf,
  plot_theme = 1,
  symb_col = "black",
  symb_fill = flclr,
  symb_size = 8,
  symb_shape = 21,
  diffint_col = "white",
  diffint_size = 1.5,
  q_line_col = "black",
  q_line_alpha = 0.8,
  q_line_size = 1.8,
  theme2_alpha = NULL
) 




dede <- psf[[1]] + geom_hline(yintercept = 0,colour=lnclr, linetype = "longdash",size=1) + ylim(-0.3,0.3)
ggsave(
  filename = paste(sel1,"vs",sel2,"_T1_22",'.png',sep=""),
  plot = dede,
  device = NULL,
  path = NULL,
  scale = 1,
  width = NA,
  height = NA,
  units = c("mm"),
  dpi = 300,
  limitsize = TRUE
)
}}
#+  scale_x_continuous(breaks = seq(6, 20, 2), limits = c(8, 20)) +  scale_y_continuous(breaks = seq(-8, 4, 2), limits = c(-8, 4))

# T1 with correction

In [ ]:
df <- tibble(N = 1:1000)
for (i in seq(from=1, to=length(sessionsB1), by=2)) {
  curRetest <- readMat(paste(derivativesDir,sessionsB1[i],"/stat/sub-invivo_",sessionsB1[i],"_desc-WM_metrics.mat",sep = ""))
  curTest <- readMat(paste(derivativesDir,sessionsB1[i+1],"/stat/sub-invivo_",sessionsB1[i+1],"_desc-WM_metrics.mat",sep = ""))
  tmp <- Map(function(x,y) mean(c(x,y)), curRetest$"T1cor", curTest$"T1cor")
  curAvg <- as.numeric(unlist(tmp))
  curAvg <- curAvg[curAvg > 0]
  curAvg <- curAvg[curAvg < 3]
  # Do not choose the same element more than once            
  curAvg <- sample (curAvg, size=1000, replace =F)          
  varName <- toString(sessionsB1[i])
  varName <- substr(varName,5,nchar(varName)-6)
  df <- mutate(df, !!varName := curAvg)       
}
             
neutral = list("rth750","rthPRI","rthSKY")
native  = list("vendorPRI","vendorSKY")
combolar <- combn(c(1,2,3),2)

for (j in seq(from=1,to=2, by=1)){
    if (j==1){
        upper = 3
    } else {
        # native
        upper = 1
    }
for (i in seq(from=1,to=upper, by=1)){

if (j==1){
    sel1 = toString(neutral[combolar[,i][1]])
    sel2 = toString(neutral[combolar[,i][2]])
    }
else
{
    sel1 = toString(native[combolar[,i][1]])
    sel2 = toString(native[combolar[,i][2]])
}
    
xx <- df[[sel1]]
yy <- df[[sel2]]

aa <- mkt2(xx,yy,group_labels=c(sel1,sel2))
sf <- shiftdhd_pbci(
  data = aa,
  formula = obs ~ gr,
  nboot = 1000,
  alpha = 0.05/30,
)

print(paste(sel1,"vs",sel2,"_T1",'.png',sep=""))    
print(min(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))
print(max(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))
    
th <- ggplot2::theme_dark() + ggplot2::theme(
  # get rid of panel grids
  #panel.grid.major = element_blank(),
  #panel.grid.minor = element_blank(),
 # legend.title = element_blank(),  
  # Change plot and panel background
  panel.border = element_blank(),
  panel.background = element_rect(fill = "black"),
  plot.background = element_rect(fill = "black"),
  legend.background = element_rect(fill="white",size=0.5),  
  legend.text=element_text(color="white",size=12),  
  #panel.border=element_blank(),   
  axis.line=element_blank(),
  #axis.ticks = element_blank(),
  axis.text = element_blank(),
    axis.title = element_blank(),
  #axis.title = element_text(colour="white",size=8),
  axis.text.x = element_text(colour="white",size=18,face='bold'),
  axis.text.y = element_text(colour="white",size=18,face='bold')
  ) 

psf <- plot_sf(
  data = sf,
  plot_theme = 1,
  symb_col = "black",
  symb_fill = NULL,
  symb_size = 8,
  symb_shape = 21,
  diffint_col = "white",
  diffint_size = 1.5,
  q_line_col = "gray",
  q_line_alpha = 0.8,
  q_line_size = 1.8,
  theme2_alpha = NULL
) 


if (j==1){
    lnclr = "#f8cbad"
} else
{
    lnclr = "#b4c7e7"
}

dede <- psf[[1]] + th + geom_hline(yintercept = 0,colour=lnclr, linetype = "longdash",size=1)
ggsave(
  filename = paste(sel1,"vs",sel2,"_T1cor",'.png',sep=""),
  plot = dede,
  device = NULL,
  path = NULL,
  scale = 1,
  width = NA,
  height = NA,
  units = c("mm"),
  dpi = 300,
  limitsize = TRUE,
  bg = "black"
)
}}
#+  scale_x_continuous(breaks = seq(6, 20, 2), limits = c(8, 20)) +  scale_y_continuous(breaks = seq(-8, 4, 2), limits = c(-8, 4))

# MTsat without correction

In [ ]:
set.seed(123)

df <- tibble(N = 1:50000)
for (i in seq(from=1, to=length(sessions), by=2)) {
  curRetest <- readMat(paste(derivativesDir,sessions[i],"/stat/sub-invivo_",sessions[i],"_desc-WM_metrics.mat",sep = ""))
  curTest <- readMat(paste(derivativesDir,sessions[i+1],"/stat/sub-invivo_",sessions[i+1],"_desc-WM_metrics.mat",sep = ""))
  tmp <- Map(function(x,y) mean(c(x,y)), curRetest$"MTsat", curTest$"MTsat")
  curAvg <- as.numeric(unlist(tmp))
  curAvg <- curAvg[curAvg > 1]
  curAvg <- curAvg[curAvg < 8]
  # Do not choose the same element more than once            
  curAvg <- sample (curAvg, size=50000, replace =F)          
  varName <- toString(sessions[i])
  varName <- substr(varName,5,nchar(varName)-6)
  df <- mutate(df, !!varName := curAvg)       
}
             
neutral = list("rth750","rthPRI","rthSKY")
native  = list("vendor750","vendorPRI","vendorSKY")
combolar <- combn(c(1,2,3),2)

for (j in seq(from=1,to=2, by=1)){
for (i in seq(from=1,to=3, by=1)){

    
if (j==1){
    sel1 = toString(neutral[combolar[,i][1]])
    sel2 = toString(neutral[combolar[,i][2]])
    }
else
{
    sel1 = toString(native[combolar[,i][1]])
    sel2 = toString(native[combolar[,i][2]])
}

print(paste(sel1,"vs",sel2,"_MTsat",' in progress',sep=""))
    
xx <- df[[sel1]]
yy <- df[[sel2]]

aa <- mkt2(xx,yy,group_labels=c(sel1,sel2))
sf <- shiftdhd_pbci(
  data = aa,
  formula = obs ~ gr,
  nboot = 250,
  alpha = 0.05/30,
)

sf[[1]]['pctdif'] = percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])
write.csv(sf,paste(sel1,"vs",sel2,"_MTsat",'.csv',sep=""))
print(min(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))
print(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])[5])
print(max(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))

    
th <- ggplot2::theme_dark() + ggplot2::theme(
  # get rid of panel grids
  #panel.grid.major = element_blank(),
  #panel.grid.minor = element_blank(),
 # legend.title = element_blank(),  
  # Change plot and panel background
  panel.border = element_blank(),
  panel.background = element_rect(fill = "black"),
  plot.background = element_rect(fill = "black"),
  legend.background = element_rect(fill="white",size=0.5),  
  legend.text=element_text(color="white",size=12),  
  #panel.border=element_blank(),   
  axis.line=element_blank(),
  #axis.ticks = element_blank(),
  axis.text = element_blank(),
    axis.title = element_blank(),
  #axis.title = element_text(colour="white",size=8),
  axis.text.x = element_text(colour="white",size=18,face='bold'),
  axis.text.y = element_text(colour="white",size=18,face='bold')
  ) 

if (j==1){
    lnclr = "darkgray"
    flclr = "red"
} else
{
    lnclr = "darkgray"
    flclr = "blue"
}
    
psf <- plot_sf(
  data = sf,
  plot_theme = 1,
  symb_col = "black",
  symb_fill = flclr,
  symb_size = 8,
  symb_shape = 21,
  diffint_col = "white",
  diffint_size = 1.5,
  q_line_col = "gray",
  q_line_alpha = 0.8,
  q_line_size = 1.8,
  theme2_alpha = NULL
) 




dede <- psf[[1]] + geom_hline(yintercept = 0,colour=lnclr, linetype = "longdash",size=1) + ylim(-1.7,1.7)
ggsave(
  filename = paste(sel1,"vs",sel2,"_MTsat_22",'.png',sep=""),
  plot = dede,
  device = NULL,
  path = NULL,
  scale = 1,
  width = NA,
  height = NA,
  units = c("mm"),
  dpi = 300,
  limitsize = TRUE,
  bg = "black"
)
}}
#+  scale_x_continuous(breaks = seq(6, 20, 2), limits = c(8, 20)) +  scale_y_continuous(breaks = seq(-8, 4, 2), limits = c(-8, 4))

# MTsat with correction

In [ ]:
df <- tibble(N = 1:1000)
for (i in seq(from=1, to=length(sessionsB1), by=2)) {
  varName <- toString(sessionsB1[i])
  varName <- substr(varName,5,nchar(varName)-6)
    
  curRetest <- readMat(paste(derivativesDir,sessionsB1[i],"/stat/sub-invivo_",sessionsB1[i],"_desc-WM_metrics.mat",sep = ""))
  curTest <- readMat(paste(derivativesDir,sessionsB1[i+1],"/stat/sub-invivo_",sessionsB1[i+1],"_desc-WM_metrics.mat",sep = ""))
  if (varName %in% neutral){
      tmp <- Map(function(x,y) mean(c(x,y)), curRetest$"MTsatCFcor", curTest$"MTsatCFcor")
  } else
  {
     tmp <- Map(function(x,y) mean(c(x,y)), curRetest$"MTsatcor", curTest$"MTsatcor")
  }
  
  curAvg <- as.numeric(unlist(tmp))
  curAvg <- curAvg[curAvg > 1]
  curAvg <- curAvg[curAvg < 6]
  # Do not choose the same element more than once            
  curAvg <- sample (curAvg, size=1000, replace =F)          
  df <- mutate(df, !!varName := curAvg)       
}
             
neutral = list("rth750","rthPRI","rthSKY")
native  = list("vendorPRI","vendorSKY")
combolar <- combn(c(1,2,3),2)

for (j in seq(from=1,to=2, by=1)){
    if (j==1){
        upper = 3
    } else {
        # native
        upper = 1
    }
for (i in seq(from=1,to=upper, by=1)){

if (j==1){
    sel1 = toString(neutral[combolar[,i][1]])
    sel2 = toString(neutral[combolar[,i][2]])
    }
else
{
    sel1 = toString(native[combolar[,i][1]])
    sel2 = toString(native[combolar[,i][2]])
}
    
xx <- df[[sel1]]
yy <- df[[sel2]]

aa <- mkt2(xx,yy,group_labels=c(sel1,sel2))
sf <- shiftdhd_pbci(
  data = aa,
  formula = obs ~ gr,
  nboot = 1000,
  alpha = 0.05/30,
)


    
th <- ggplot2::theme_dark() + ggplot2::theme(
  # get rid of panel grids
  #panel.grid.major = element_blank(),
  #panel.grid.minor = element_blank(),
 # legend.title = element_blank(),  
  # Change plot and panel background
  panel.border = element_blank(),
  panel.background = element_rect(fill = "black"),
  plot.background = element_rect(fill = "black"),
  legend.background = element_rect(fill="white",size=0.5),  
  legend.text=element_text(color="white",size=12),  
  #panel.border=element_blank(),   
  axis.line=element_blank(),
  #axis.ticks = element_blank(),
  axis.text = element_blank(),
    axis.title = element_blank(),
  #axis.title = element_text(colour="white",size=8),
  axis.text.x = element_text(colour="white",size=18,face='bold'),
  axis.text.y = element_text(colour="white",size=18,face='bold')
  ) 

psf <- plot_sf(
  data = sf,
  plot_theme = 1,
  symb_col = "black",
  symb_fill = NULL,
  symb_size = 8,
  symb_shape = 21,
  diffint_col = "white",
  diffint_size = 1.5,
  q_line_col = "gray",
  q_line_alpha = 0.8,
  q_line_size = 1.8,
  theme2_alpha = NULL
) 


if (j==1){
    lnclr = "#f8cbad"
} else
{
    lnclr = "#b4c7e7"
}

dede <- psf[[1]] + th + geom_hline(yintercept = 0,colour=lnclr, linetype = "longdash",size=1)
ggsave(
  filename = paste(sel1,"vs",sel2,"_MTsatcor",'.png',sep=""),
  plot = dede,
  device = NULL,
  path = NULL,
  scale = 1,
  width = NA,
  height = NA,
  units = c("mm"),
  dpi = 300,
  limitsize = TRUE,
  bg = "black"
)
}}
#+  scale_x_continuous(breaks = seq(6, 20, 2), limits = c(8, 20)) +  scale_y_continuous(breaks = seq(-8, 4, 2), limits = c(-8, 4))

## MTR 

In [ ]:
set.seed(123)
df <- tibble(N = 1:50000)
for (i in seq(from=1, to=length(sessions), by=2)) {
  curRetest <- readMat(paste(derivativesDir,sessions[i],"/stat/sub-invivo_",sessions[i],"_desc-WM_metrics.mat",sep = ""))
  curTest <- readMat(paste(derivativesDir,sessions[i+1],"/stat/sub-invivo_",sessions[i+1],"_desc-WM_metrics.mat",sep = ""))
  tmp <- Map(function(x,y) mean(c(x,y)), curRetest$"MTR", curTest$"MTR")
  curAvg <- as.numeric(unlist(tmp))
  curAvg <- curAvg[curAvg > 35]
  curAvg <- curAvg[curAvg < 70]
  # Do not choose the same element more than once            
  curAvg <- sample (curAvg, size=50000, replace =F)          
  varName <- toString(sessions[i])
  varName <- substr(varName,5,nchar(varName)-6)
  df <- mutate(df, !!varName := curAvg)       
}
             
neutral = list("rth750","rthPRI","rthSKY")
native  = list("vendor750","vendorPRI","vendorSKY")
combolar <- combn(c(1,2,3),2)

for (j in seq(from=1,to=2, by=1)){
for (i in seq(from=1,to=3, by=1)){

if (j==1){
    sel1 = toString(neutral[combolar[,i][1]])
    sel2 = toString(neutral[combolar[,i][2]])
    }
else
{
    sel1 = toString(native[combolar[,i][1]])
    sel2 = toString(native[combolar[,i][2]])
}
    
xx <- df[[sel1]]
yy <- df[[sel2]]

aa <- mkt2(xx,yy,group_labels=c(sel1,sel2))
sf <- shiftdhd_pbci(
  data = aa,
  formula = obs ~ gr,
  nboot = 250,
  alpha = 0.05/30,
)

sf[[1]]['pctdif'] = percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])
write.csv(sf,paste(sel1,"vs",sel2,"_MTR",'.csv',sep=""))
print(min(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))
print(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])[5])
print(max(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])))
    
th <- ggplot2::theme_dark() + ggplot2::theme(
  # get rid of panel grids
  #panel.grid.major = element_blank(),
  #panel.grid.minor = element_blank(),
 # legend.title = element_blank(),  
  # Change plot and panel background
  panel.border = element_blank(),
  panel.background = element_rect(fill = "black"),
  plot.background = element_rect(fill = "black"),
  legend.background = element_rect(fill="white",size=0.5),  
  legend.text=element_text(color="white",size=12),  
  #panel.border=element_blank(),   
  axis.line=element_blank(),
  #axis.ticks = element_blank(),
  axis.text = element_blank(),
    axis.title = element_blank(),
  #axis.title = element_text(colour="white",size=8),
  axis.text.x = element_text(colour="white",size=18,face='bold'),
  axis.text.y = element_text(colour="white",size=18,face='bold')
  ) 

if (j==1){
    lnclr = "darkgray"
    flclr = "red"
} else
{
    lnclr = "darkgray"
    flclr = "blue"
}
    
    
psf <- plot_sf(
  data = sf,
  plot_theme = 1,
  symb_col = "black",
  symb_fill = flclr,
  symb_size = 8,
  symb_shape = 21,
  diffint_col = "white",
  diffint_size = 1.5,
  q_line_col = "gray",
  q_line_alpha = 0.8,
  q_line_size = 1.8,
  theme2_alpha = NULL
) 


dede <- psf[[1]]  + geom_hline(yintercept = 0,colour=lnclr, linetype = "longdash",size=1) + ylim(-6,6)
ggsave(
  filename = paste(sel1,"vs",sel2,"_MTR_22",'.png',sep=""),
  plot = dede,
  device = NULL,
  path = NULL,
  scale = 1,
  width = NA,
  height = NA,
  units = c("mm"),
  dpi = 300,
  limitsize = TRUE,
  bg = "black"
)
}}
#+  scale_x_continuous(breaks = seq(6, 20, 2), limits = c(8, 20)) +  scale_y_continuous(breaks = seq(-8, 4, 2), limits = c(-8, 4))

In [ ]:
names(sf)

In [ ]:
names(sf[1])

In [ ]:
aaa = sf$`vendorPRI - vendorSKY`$'vendorPRI'
bbb = sf$`vendorPRI - vendorSKY`$'vendorSKY'

percentage_difference(aaa,bbb)
percentage_difference(aaa,bbb)

In [ ]:
aaa

In [ ]:
bbb

In [ ]:
of = paste('`',sel1,'-',sel2,'`',sep="")

In [ ]:
of = "`vendorPRI - vendorSKY`"

In [ ]:
aa <- percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])

In [ ]:
print(percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])[4])

In [ ]:
percentage_difference(sf[[1]][[sel1]],sf[[1]][[sel2]])

In [ ]:
write.csv(sf,'anan.csv')

In [ ]:
sf[[1]]

In [ ]:
dede + ylim(-1.1,1.1)